In [10]:
import pandas as pd
import geopandas as gpd
import os

In [15]:
#path to read

path_to_alt='/home/angelica/Documents/py_linux/pylocss/data/karina/ourlakes/tmb_lakes/'
# open lakes
path_lakes='/home/angelica/Dropbox/Research/LOCSS/other_lakes/Selected_lakes/all_lakes_paper.shp'
df_sel_lakes=gpd.read_file(path_lakes)
df_sel_lakes.loc[df_sel_lakes['lake_id']=='9129']#.unique()

,lake_name,Country,Lake_area,gauge_id,source,Computed,lake_id,arae,max_area,lake_from,ol_Aproved,geometry
25,Chautauqua Lake,United States of America,53.34,CQK2,LOCSS,Y,9129,NaN,NaN,HLK,3.0,"POLYGON ((-79.47465 42.25947, -79.47348 42.258..."


In [16]:
os.listdir(path_to_alt)

['19069',
 '7320414542',
 '4530720592',
 '12030',
 '1052515',
 '12327',
 '7320415112',
 '12036',
 '6610149612',
 '7421066862',
 '7830177462',
 '7320428982',
 '7320417422',
 '21',
 '6530064712',
 '7320103213',
 '19060',
 '6610142963',
 '19126',
 '7820034152',
 '22',
 '12056',
 '1052865',
 '865120',
 '6620000613']

In [26]:
col_id_fd='lake_id'
list_lakes=df_sel_lakes.loc[df_sel_lakes['source'].isin(['RVBR', 'LOCSS', 'ARNH', 'USGS']),col_id_fd].unique()
#TODO validate empty list
df_files=pd.DataFrame()
sep=' '

for folder in os.listdir(path_to_alt):
        path=path_to_alt+folder+'/'
        files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))&(f.endswith('ocog.dat'))]
        print(files) 
        for f in files:
            f_name=path+f
            df_file=pd.read_csv(f_name, sep=sep)
            df_file['sat']=f[:3]
            df_file['orbit']=f[4:7]
            df_file['lake_id']=folder
            df_files=pd.concat([df_files, df_file], axis=0)
            

['S3A_095_ocog.dat', 'S3B_095_ocog.dat']
         time  track      height        lat        lon  OCval  cycle  sat  \
0    2016.288      2  661.800982 -27.478393 -51.619109     61      3  S3A   
1    2016.288      2  648.566358 -27.492954 -51.622941     95      3  S3A   
2    2016.288      2  705.514028 -27.495866 -51.623708     30      3  S3A   
3    2016.288      2  710.160486 -27.524983 -51.631374     60      3  S3A   
4    2016.288      2  698.109376 -27.527895 -51.632141     60      3  S3A   
..        ...    ...         ...        ...        ...    ...    ...  ...   
736  2023.607     99  471.840693 -27.528465 -51.629552     61    102  S3A   
737  2023.607     99  476.242212 -27.531378 -51.630319     62    102  S3A   
738  2023.607     99  476.344525 -27.534292 -51.631087     62    102  S3A   
739  2023.607     99  476.129497 -27.537204 -51.631854     94    102  S3A   
740  2023.607     99  476.196641 -27.540118 -51.632621     87    102  S3A   

    orbit lake_id  
0     095   19